In [1]:
import pipeline_perso 

In [59]:
import numpy as np

In [60]:
a=np.array([5,2,6,9,8,8,9])

In [65]:
a[np.where(a==8)]=3

In [66]:
a

array([5, 6, 6, 9, 3, 3, 9])

In [67]:
!pwd

/home/mohamed/Desktop/Projet_IML/Mini-projet-IML


In [166]:
import pandas as pd
d=pd.read_csv('DigiDB_digimonlist.csv')

In [179]:
a=d[6:9]

In [180]:
a

,Number,Digimon,Stage,Type,Attribute,Memory,Equip Slots,Lv 50 HP,Lv50 SP,Lv50 Atk,Lv50 Def,Lv50 Int,Lv50 Spd
6,7,Tanemon,In-Training,Free,Plant,3,0,1030,64,85,82,73,69
7,8,Tsunomon,In-Training,Free,Earth,3,0,930,54,107,92,54,76
8,9,Tsumemon,In-Training,Free,Dark,3,0,930,64,108,64,54,93


In [181]:
a1=np.array(a)

In [192]:
a1[:,8]

array([64, 54, 64], dtype=object)

In [204]:
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    """Returns data with categorical features handled following a strategy given as argument.

    Arguments and function description

    @author : HAMDI Mohamed
    """

    # Class constructor method that takes in a list of values as its argument
    def __init__(self, strategy, categorical_cols_names=None):
        self._strategy = strategy
        self._categorical_vars = categorical_cols_names

    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self

    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):

        X = pd.DataFrame(X).convert_dtypes()
        if self._categorical_vars is None:
            self._categorical_vars = list(X.select_dtypes(include=['string']).columns)
        for col in self._categorical_vars:
            categories = list(X[col].drop_duplicates())
            if self._strategy == "one_hot_encoding":
                aux = pd.get_dummies(X[col], prefix=col)
                X = pd.concat([X, aux], axis=1)
                X = X.drop([col], axis=1)
            if self._strategy == "ordinal_encoding":
                replacement_dic = {categories[i]: i for i in range(len(categories))}
                X[col] = X[col].replace(replacement_dic)
            if self._strategy == "target_encoding":
                aux = pd.DataFrame({'col': X[col], 'target': y})
                aux = aux.groupby(['col']).mean(numeric_only=False)
                replacement_dic = dict(zip(aux.index, aux.target))
                X[col] = X[col].replace(replacement_dic)
        return X

In [210]:
t=CategoricalTransformer(strategy="target_encoding",categorical_cols_names=[1,2,3])

In [211]:
pd.DataFrame(a1)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7,Tanemon,In-Training,Free,Plant,3,0,1030,64,85,82,73,69
1,8,Tsunomon,In-Training,Free,Earth,3,0,930,54,107,92,54,76
2,9,Tsumemon,In-Training,Free,Dark,3,0,930,64,108,64,54,93


In [212]:
t.transform(a1,y=a1[:,8])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7,64.0,60.6667,60.6667,Plant,3,0,1030,64,85,82,73,69
1,8,54.0,60.6667,60.6667,Earth,3,0,930,54,107,92,54,76
2,9,64.0,60.6667,60.6667,Dark,3,0,930,64,108,64,54,93
